In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install PyPDF2 sentence-transformers faiss-cpu 



   -------------------------------------- 232.6/232.6 kB 710.3 kB/s eta 0:00:00
   -------------------------------------- 470.2/470.2 kB 866.2 kB/s eta 0:00:00
   ---------------------------------------- 15.0/15.0 MB 642.6 kB/s eta 0:00:00



[notice] A new release of pip is available: 23.3 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
!pip install PyMuPDF

   ---------------------------------------- 18.7/18.7 MB 897.8 kB/s eta 0:00:00



[notice] A new release of pip is available: 23.3 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


## Load a Board Game Rulebook and Extract Text

In [9]:
import fitz  # PyMuPDF

file_path = "ideabase_chessrules.pdf"  

doc = fitz.open(file_path)

full_text = ""
for page_num,page in enumerate(doc):
    page_text = page.get_text()
    full_text += f"\n--- Page {page_num + 1} ---\n" + page_text

print(full_text[:5000])  # Print the first 1000 characters to check the content
    


--- Page 1 ---
3
THE OFFICIAL RULES OF CHESS • CARDOZA PUBLISHING
Rules of
Chess
ERIC SCHILLER

--- Page 2 ---
THE OFFICIAL RULES OF CHESS • CARDOZA PUBLISHING
THE OFFICIAL RULES OF CHESS
The following are the standard rules of chess as applied in World
Championship competition. In later chapter we present some of the
variations of the rules used in amateur, scholastic, and online com-
petitions. These rules conform in most part to those of the world
chess federation (FIDE), but differ significantly from those found in
American tournaments conducted under the auspices of the United
States Chess Federation. Since most American tournaments are
amateur events, those rules are discussed in the section on ama-
teur rules.
This set of rules was composed by International Arbiter Eric
Schiller, with the cooperation and valuable assistance of Interna-
tional Arbiters Andrzej Filipowicz (Poland) and Yuri Averbakh (Rus-
sia). They were used verbatim in the 2000 Braingames.net World
Chess Champio

### Clean and Preprocess the Rulebook Text

In [10]:
import re

# Step 1: Remove long sequences of newlines (convert to 1 newline)
cleaned_text = re.sub(r'\n{2,}', '\n', full_text)

# Step 2: Remove bullet characters or other non-standard symbols
cleaned_text = re.sub(r'[•·►\uf0b7\xa0]', '', cleaned_text)

# Step 3: Remove common footer/header artifacts (if you notice any patterns like "Page x" or "FIDE Laws of Chess")
cleaned_text = re.sub(r'Page\s+\d+', '', cleaned_text, flags=re.IGNORECASE)

# Step 4: Strip extra whitespace from beginning and end of each line
cleaned_text = '\n'.join([line.strip() for line in cleaned_text.splitlines()])

# Step 5: (Optional) Convert to lowercase for normalization
cleaned_text = cleaned_text.lower()  # Only if you're okay making everything lowercase

# Preview the cleaned result
print(cleaned_text[:1500])



---  ---
3
the official rules of chess  cardoza publishing
rules of
chess
eric schiller
---  ---
the official rules of chess  cardoza publishing
the official rules of chess
the following are the standard rules of chess as applied in world
championship competition. in later chapter we present some of the
variations of the rules used in amateur, scholastic, and online com-
petitions. these rules conform in most part to those of the world
chess federation (fide), but differ significantly from those found in
american tournaments conducted under the auspices of the united
states chess federation. since most american tournaments are
amateur events, those rules are discussed in the section on ama-
teur rules.
this set of rules was composed by international arbiter eric
schiller, with the cooperation and valuable assistance of interna-
tional arbiters andrzej filipowicz (poland) and yuri averbakh (rus-
sia). they were used verbatim in the 2000 braingames.net world
chess championship and were 

In [11]:
print(cleaned_text[:5000])  # Print the first 5000 characters to check the cleaned content


---  ---
3
the official rules of chess  cardoza publishing
rules of
chess
eric schiller
---  ---
the official rules of chess  cardoza publishing
the official rules of chess
the following are the standard rules of chess as applied in world
championship competition. in later chapter we present some of the
variations of the rules used in amateur, scholastic, and online com-
petitions. these rules conform in most part to those of the world
chess federation (fide), but differ significantly from those found in
american tournaments conducted under the auspices of the united
states chess federation. since most american tournaments are
amateur events, those rules are discussed in the section on ama-
teur rules.
this set of rules was composed by international arbiter eric
schiller, with the cooperation and valuable assistance of interna-
tional arbiters andrzej filipowicz (poland) and yuri averbakh (rus-
sia). they were used verbatim in the 2000 braingames.net world
chess championship and were 